Named Entity Recognition with recent and improved version of the Annotated NER Dataset with better features

**Note: It is just a quick demonstration on 'How to use this dataset on Scikit for Named Entity Classification'**
**This is not for analytical concepts and other machine learning illustrations!**

Reading CSV in pandas has many ways. I suggest to use CSV as database to prevent large file handling errors.
One may use the follwing way to read CSV file using Python Pandas. Good Luck

In [1]:
!pip install -U keras

  Using cached https://files.pythonhosted.org/packages/ad/fd/6bfe87920d7f4fd475acd28500a42482b6b84479832bdc0fe9e589a60ceb/Keras-2.3.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882ce099b03f/Keras_Applications-1.0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/28/6a/8c1f62c37212d9fc441a7e26736df51ce6f0e38455816445471f10da4f0a/Keras_Preprocessing-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e1/63/d919e16c5bd3502a0f7675f217625bd6f49a412cc1a856aa6b4b5b5b20bc/scipy-1.3.1-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/0b/fa/bee65d2dbdbd3611702aafd128139c53c90a1285f169ba5467aab252e27a/h5py-2.10.0-cp36-cp36m-win_amd64.whl


You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
pip install --upgrade wandb

Requirement already up-to-date: wandb in c:\users\jeremy\documents\code\ml-tools-and-examples\venv\lib\site-packages (0.8.14)


You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
pip install tensorflow-gpu

  Using cached https://files.pythonhosted.org/packages/d5/cf/83a8b5aa711adcdbed5496618663ed99878b28e843593cb6b2fae6a59181/tensorflow_gpu-2.0.0-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/8a/48/a76be51647d0eb9f10e2a4511bf3ffb8cc1e6b14e9e4fab46173aa79f981/termcolor-1.1.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/d3/9e/a48cd34dd7b672ffc227b566f7d16d63c62c58b542d54efa45848c395dd4/tensorboard-2.0.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/23/84/323c2415280bc4fc880ac5050dddfb3c8062c2552b34c2e512eb4aa68f79/wrapt-1.11.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/4e/35/11749bf99b2d4e3cceb4d55ca22590b0d7c2c62b9de38ac4a4a7f4687421/gast-0.2.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/fc/08/8b927337b7019c374719145d1dceba21a8bb909b93b1ad6f8fb7d22c1ca1/tensorflow_estimator-2.0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3b/72/e6e483e2db953c11e

You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
!wandb login 2295caae2022fe6a431a755f74d1e9e8605848eb
import wandb
from wandb.keras import WandbCallback
wandb.init(project="16-class-ner")


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Jeremy/.netrc
wandb: WARNING import wandb.keras called before import keras or import tensorflow.keras.  This can lead to a version mismatch, W&B now assumes tensorflow.keras


Successfully logged in to Weights & Biases!


W&B Run: https://app.wandb.ai/jeremy/16-class-ner/runs/9qarv1gw

In [7]:
pip install pandas scikit-learn numpy

  Using cached https://files.pythonhosted.org/packages/f0/ac/92c3d2f0b627efbd1a7b2156faa697f9c2bbd7b0fe83ba8a9d36f982156f/pandas-0.25.3-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/76/79/60050330fe57fb59f2c53d0d11673df28c20ea9315da3652477429fc4949/scikit_learn-0.21.3-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/e7/f9/f0b53f88060247251bf481fa6ea62cd0d25bf1b11a87888e53ce5b7c8ad2/pytz-2019.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8f/42/155696f85f344c066e17af287359c9786b436b1bf86029bb3411283274f3/joblib-0.14.0-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [8]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, f1_score

In [9]:
import keras
from keras.preprocessing.text import *
from keras.preprocessing import *
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MultiLabelBinarizer, LabelEncoder

Using TensorFlow backend.


In [10]:
df = pd.read_csv("D:\\Data\\entity-annotated-corpus\\ner.csv", encoding = "ISO-8859-1", error_bad_lines=False)
df.dropna(inplace=True)

b'Skipping line 281837: expected 25 fields, saw 34\n'


In [12]:
print(len(df['sentence_idx'].unique()))
num_samples = len(df)
num_samples

35177


1050794

In [13]:
filtered_df = df.filter(['word', 'tag', 'sentence_idx'])
filtered_df.tail()

,word,tag,sentence_idx
1050790,they,O,47959.0
1050791,responded,O,47959.0
1050792,to,O,47959.0
1050793,the,O,47959.0
1050794,attack,O,47959.0


In [14]:
def get_multilabel():
  sentences = []
  tags = []

  sentids = filtered_df['sentence_idx'].unique()

  curr_sent_id = filtered_df['sentence_idx'].iloc[0]
  tokens = []
  tag_tokens = []
  row = None
  for index, row in filtered_df.iterrows():

    sent_id = row['sentence_idx']

    if curr_sent_id == sent_id:
      tokens.append(row['word'])
      tag_tokens.append(row['tag'])
    else:
      curr_sent_id = sent_id
      sentences.append(" ".join(tokens))
      tags.append(tag_tokens)
      tokens = []
      tag_tokens = []

  # get the last sentence
  sentences.append(" ".join(tokens))
  tags.append(tag_tokens)# print(sentences)
  
  x, y, enc, tok = prepare_multilabel(sentences, tags)
  
  return x, y, enc, tok

def prepare_multilabel(sentences, tags):
  # prepare y data
  y = np.array(tags)
  enc = MultiLabelBinarizer()
  y = enc.fit_transform(y)
  print(enc.classes_)
  print(y[0])
  print(y.shape)
  
  # prepare x data
  x = np.array(sentences)
  tok = Tokenizer()
  tok.fit_on_texts(x)
  x = tok.texts_to_sequences(x)
  x = sequence.pad_sequences(x, maxlen=100, padding='post')
  x = np.array(x)
  return x, y, enc, tok


In [17]:
def get_singlelabel():
  words = filtered_df['word'].values
  tags = filtered_df['tag'].values
  
  x, y, le,enc, tok = prepare_singlelabel(words, tags)
  return x, y, le, enc, tok
  
def prepare_singlelabel(words, tags):
  
  # prepare x
  x = np.array(words)
  tok = Tokenizer()
  tok.fit_on_texts(x)
  x = tok.texts_to_sequences(x)
  x = sequence.pad_sequences(x, maxlen=1, padding='post')
  x = np.array(x)
    
  # prepare y
  le = LabelEncoder()
  labels = le.fit_transform(tags)
  enc = LabelBinarizer()
  y = enc.fit_transform(labels)
  
  print(enc.classes_)
  print(y[0])
  print(y.shape)
  
  return x, y, le, enc, tok

In [18]:
# x, y = get_multilabel()
# x, y, le, enc, tok = get_singlelabel()
x,y,le,enc,tok = get_singlelabel()

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
(1050794, 17)


In [0]:
pip install dill

In [0]:
import dill
with open("data_utils.pkl", "wb") as f:
  dill.dump((x,y,le,enc,tok), f)

In [19]:
print(len(x))
print(len(y))

1050794
1050794


In [20]:
 # print(tok.__dict__.keys())
num_words = len(tok.word_index)

In [21]:
num_words

24381

In [22]:
# classes = filtered_df['tag'].unique()
num_classes = y.shape[1]
num_classes

17

Splitting Train and test data

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
print(x_train.shape)
print(y_train.shape)

(735555, 1)
(735555, 17)


In [0]:
def hyperas_data_fn():
  import dill
  with open("data_utils.pkl", "rb") as f:
    x,y,le,enc,tok = dill.load(f)
    return x,y,le,enc,tok 

In [24]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, GlobalMaxPooling1D
from keras.optimizers import SGD
from keras import backend as K
  

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

  
def get_model(num_classes, num_words, embed_out=64, dense_1_nodes=64, dropout_1=0.5):
    model = Sequential()
    model.add(Embedding(num_words+1, embed_out, input_length=1, name='embedding_layer'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(dense_1_nodes, activation='relu', name='dense_1'))
    model.add(Dropout(dropout_1))
#     model.add(Dense(num_classes, activation='sigmoid')) # multilabel
    model.add(Dense(num_classes, activation='sigmoid', name='output_layer')) #single label
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
    return model

In [27]:
import random
batch_size = 1024


def run_trials_single():
  run_name = "Testing new GPU" # what are you testing with this run
  notes = ""

  #hyperparams
  hyperparams = dict(
    embed_out =random.randint(8,256),
    dense_1_nodes = random.randint(8,256),
    dropout_1=random.uniform(0,0.95),
  )

  config= {
      "epochs": 50,
      "num_classes":num_classes,
      "num_words": num_words,
      "batch_size":batch_size,
      **hyperparams
  }

  model = get_model(num_classes, num_words, **hyperparams)

  wandb.init(project="16-class-ner", name=run_name, config=config, notes=notes)

  history = model.fit(x_train, y_train, batch_size=config['batch_size'], epochs=config['epochs'], 
            validation_split=0.2, callbacks=[WandbCallback()])

  scores = model.evaluate(x_test, y_test)
  wandb.log({'test_'+metric: scores[index] for index, metric in enumerate(model.metrics_names)})
  print("\n*** Finished! ***")

In [ ]:
for x in range(0,100):
  run_trials_single()

c:\users\jeremy\documents\code\ml-tools-and-examples\venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 588444 samples, validate on 147111 samples
Epoch 1/50
588444/588444 [==============================] - 3s 6us/step - loss: 0.6139 - acc: 0.8847 - f1_m: 0.8547 - precision_m: 0.8352 - recall_m: 0.9036 - val_loss: 0.2427 - val_acc: 0.9324 - val_f1_m: 0.9308 - val_precision_m: 0.9305 - val_recall_m: 0.9311
Epoch 2/50
588444/588444 [==============================] - 3s 5us/step - loss: 0.2452 - acc: 0.9320 - f1_m: 0.9270 - precision_m: 0.9314 - recall_m: 0.9226 - val_loss: 0.2261 - val_acc: 0.9354 - val_f1_m: 0.9330 - val_precision_m: 0.9381 - val_recall_m: 0.9280
Epoch 3/50
588444/588444 [==============================] - 3s 5us/step - loss: 0.2227 - acc: 0.9377 - f1_m: 0.9282 - precision_m: 0.9377 - recall_m: 0.9190 - val_loss: 0.2237 - val_acc: 0.9365 - val_f1_m: 0.9317 - val_precision_m: 0.9389 - val_recall_m: 0.9246
Epoch 4/50
588444/588444 [==============================] - 3s 5us/step - loss: 0.2137 - acc: 0.9395 - f1_m: 0.9263 - precision_m: 0.9392 - recall_m: 0.9137 - val

Epoch 32/50
588444/588444 [==============================] - 3s 5us/step - loss: 0.1852 - acc: 0.9432 - f1_m: 0.9054 - precision_m: 0.9715 - recall_m: 0.8477 - val_loss: 0.2425 - val_acc: 0.9345 - val_f1_m: 0.9035 - val_precision_m: 0.9706 - val_recall_m: 0.8452
Epoch 33/50
588444/588444 [==============================] - 3s 5us/step - loss: 0.1842 - acc: 0.9432 - f1_m: 0.9050 - precision_m: 0.9715 - recall_m: 0.8472 - val_loss: 0.2439 - val_acc: 0.9342 - val_f1_m: 0.9032 - val_precision_m: 0.9704 - val_recall_m: 0.8447
Epoch 34/50
588444/588444 [==============================] - 3s 5us/step - loss: 0.1842 - acc: 0.9434 - f1_m: 0.9055 - precision_m: 0.9714 - recall_m: 0.8480 - val_loss: 0.2440 - val_acc: 0.9347 - val_f1_m: 0.9043 - val_precision_m: 0.9702 - val_recall_m: 0.8468
Epoch 35/50
588444/588444 [==============================] - 3s 5us/step - loss: 0.1838 - acc: 0.9432 - f1_m: 0.9057 - precision_m: 0.9714 - recall_m: 0.8483 - val_loss: 0.2442 - val_acc: 0.9342 - val_f1_m: 0.90

In [0]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform


def run_trials():
  run_name = ""

  hyperparams = dict(
    embed_out =32,
    dense_1_nodes = 24,
    dropout_1=0.3,
  )

  config= {
      "epochs": 50,
      "num_classes":17,
      "num_words": 14950,
      "batch_size":{{choice([64, 128])}},
      **hyperparams
  }
  
  model = Sequential()
  model.add(Embedding(num_words+1, {{uniform(10,256)}}, input_length=1, name='embedding_layer'))
  model.add(GlobalMaxPooling1D())
  model.add(Dense({{uniform(10,128)}}, activation={{choice(['relu', 'sigmoid'])}}, name='dense_1'))
  model.add(Dropout({{uniform(10,256)}}))
  model.add(Dense(num_classes, activation='sigmoid', name='output_layer'))
  model.compile(loss='categorical_crossentropy', optimizer={{choice(['rmsprop', 'adam', 'sgd'])}}, metrics=['acc',f1_m,precision_m, recall_m])
  
  model = get_model(num_classes, num_words, **hyperparams)

  wandb.init(project="16-class-ner", name=run_name, config=config, notes=notes)

  history = model.fit(x_train, y_train, batch_size=config['batch_size'], epochs=config['epochs'], validation_split=0.2, callbacks=[WandbCallback()])

  scores = model.evaluate(x_test, y_test)
  wandb.log({'test_'+metric: scores[index] for index, metric in enumerate(model.metrics_names)})
  print("\n*** Finished! ***")

  validation_acc = np.amax(history.history['val_acc']) 
  print('Best validation acc of epoch:', validation_acc)
  return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

best_run, best_model = optim.minimize(model=run_trials, data=hyperas_data_fn, algo=tpe.suggest, max_evals=100, trials=Trials(),notebook_name='16_class_ner(1)')

>>> Imports:
#coding=utf-8

try:
    import wandb
except:
    pass

try:
    from wandb.keras import WandbCallback
except:
    pass

try:
    import json
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from sklearn.feature_extraction import DictVectorizer
except:
    pass

try:
    from sklearn.linear_model import Perceptron
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.metrics import precision_recall_fscore_support, f1_score
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.preprocessing.text import *
except:
    pass

try:
    from keras.preprocessing import *
except:
    pass

try:
    from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MultiLabelBinarizer, LabelEncoder
except:
    pass

try:
    import dill
except:
    pass

try:
    import dill
except:
    pass

try:
    import keras
except:
    p

IndexError: ignored

In [0]:
!pip install hyperas

     |████████████████████████████████| 337kB 7.4MB/s 
ERROR: ipython 5.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.4, but you'll have prompt-toolkit 2.0.9 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.16
    Uninstalling prompt-toolkit-1.0.16:
      Successfully uninstalled prompt-toolkit-1.0.16


In [0]:
test_input = tok.texts_to_sequences(['prime'])
print(test_input)
pred = model.predict(test_input)
np.argmax(pred)

enc.__dict__

[[99]]


{'_cached_dict': None,
 'classes': None,
 'classes_': array(['-', 'B', 'I', 'O', 'a', 'e', 'g', 'i', 'm', 'n', 'o', 'p', 'r',
        't', 'v'], dtype=object),
 'sparse_output': False}